# Simple Competitive Learning Model

In [16]:
import numpy as np
class SCLM:
    def __init__(self, learning_coef = 0.5, weight_vectors = [[]], training_data = [[]], testing_data = [[]]):
        self.learning_coef = learning_coef
        self.weight_vectors = np.array(weight_vectors)
        self.training_data = np.array(training_data)
        self.testing_data = np.array(testing_data)
        self.testing_labels = [f'testing_data-{idx+1}' for idx in range(len(testing_data))]
        self.training_labels = [f'training_data-{idx+1}' for idx in range(len(training_data))]
        self.init()
        

    def init(self):
        self.input_unit_labels = [f'input_unit-{idx+1}' for idx in range(self.weight_vectors.shape[1])]
        self.output_unit_labels = [f'output_unit-{idx+1}' for idx in range(self.weight_vectors.shape[0])]
        self.training_clusters = {idx: [] for idx in range(self.weight_vectors.shape[0])}
        self.testing_clusters = {idx: [] for idx in range(self.weight_vectors.shape[0])}

    
    def init_rand_weights(self, num_input_units, num_output_units):
        self.weight_vectors = self.normalize(np.random.rand(num_output_units, num_input_units))
        self.init()
        
    def normalize(self, vectors, single = False):
        if single: 
            # print((vectors / vectors.sum()).sum())
            return  vectors / vectors.sum()
        else:
            # print((vectors / vectors.sum(axis = 1)[:, np.newaxis]).sum(axis = 1))
            return  vectors / vectors.sum(axis = 1)[:, np.newaxis]

    def training_process(self):
        for idx, vector in enumerate(self.training_data):
            # Find the winning output unit (zero-base)
            winning_output_unit_idx = np.argmax(np.multiply(self.weight_vectors, vector).sum(axis = 1))
            # Clustering
            self.training_clusters[winning_output_unit_idx].append(idx)
            # helper_print_output_unit_calculations
            self.helper_print_training_process(idx, vector, winning_output_unit_idx)
            # Update the weight of winning output unit
            self.weight_vectors[winning_output_unit_idx] = self.normalize(self.weight_vectors[winning_output_unit_idx] + self.learning_coef * vector, True)
        
            print('\tUpdated Weights: \n\t\t' + self.print_current_weight().replace('\n','\n\t\t') + '\n\n')
        self.helper_print_clusters(self.training_clusters)

    def testing_process(self):
        for idx, vector in enumerate(self.testing_data):
            winning_output_unit_idx = np.argmax(np.multiply(self.weight_vectors, vector).sum(axis = 1))
            # Clustering
            self.testing_clusters[winning_output_unit_idx].append(idx)
            # helper_print_testing_process
            self.helper_print_testing_process(idx, vector, winning_output_unit_idx)
        self.helper_print_clusters(self.testing_clusters)
        
    def __str__(self):
        def helper(labels, data): return '\n'.join([f'{label}: \t{item}' for label, item in zip(labels, data)])

        return "learning_coef: " + str(self.learning_coef) + "\n"\
            "weight_vectors: \n\t" + self.print_current_weight().replace('\n','\n\t') + "\n"\
            "training_data: \n\t" + helper(self.training_labels, self.training_data).replace('\n','\n\t') + "\n"\
            "training_data: \n\t" + helper(self.testing_labels, self.testing_data).replace('\n','\n\t') + "\n"\
        # input_unit_labels: {self.input_unit_labels}
        # output_unit_labels: {self.output_unit_labels}
        # """
    def print_current_weight(self):
        def helper(labels, data): return '\n'.join([f'{label}: \t{item}' for label, item in zip(labels, np.round(data, 4))])
        return helper(self.output_unit_labels, self.weight_vectors)

#####################################################################################################
    def helper_print_testing_process(self, idx, vector, winning_output_unit_idx):
        round_dec = 4
        text = []

        # Find the winning output unit (zero-base)
        text.append(f'The Testing Process: {self.testing_labels[idx]}: {vector}\n')
        text.append(f'\tFinding the winning output unit: \n')
        for r in range(self.weight_vectors.shape[0]):
            text.append(f'\t\t{self.output_unit_labels[r]}: ')
            for c in range(self.weight_vectors.shape[1]):
                text.append(f'({round(self.weight_vectors[r][c], round_dec)}*{vector[c]})')
                text.append(' + ')
            text.pop()
            text.append(f' = {round(np.multiply(self.weight_vectors[r], vector).sum(), round_dec)}\n')
        text.append(f'\n\t\tThe winning output unit is {self.output_unit_labels[winning_output_unit_idx]}\n\n')
        text.append(f'\t{self.training_labels[idx]} clusters under {self.output_unit_labels[winning_output_unit_idx]}\n\n')        
        print(''.join(text))

    def helper_print_training_process(self, idx, vector, winning_output_unit_idx):
        round_dec = 4
        text = []

        # Find the winning output unit (zero-base)
        text.append(f'The Training Process: {self.training_labels[idx]}: {vector}\n')
        text.append(f'\tFinding the winning output unit: \n')
        for r in range(self.weight_vectors.shape[0]):
            text.append(f'\t\tThe net weighted sum of {self.output_unit_labels[r]}: ')
            for c in range(self.weight_vectors.shape[1]):
                text.append(f'({round(self.weight_vectors[r][c], round_dec)}*{vector[c]})')
                text.append(' + ')
            text.pop()
            text.append(f' = {round(np.multiply(self.weight_vectors[r], vector).sum(), round_dec)}\n')
        text.append(f'\n\t\tThe winning output unit is {self.output_unit_labels[winning_output_unit_idx]}\n\n')

        #Updating the weight of the winning output
        text.append(f'\tUpdating the weight of the winning output:\n')
        text.append(f'\t\t[')
        for c in range(self.weight_vectors.shape[1]):
            text.append(f'{round(self.weight_vectors[winning_output_unit_idx][c], round_dec)} + {self.learning_coef} * {vector[c]}')
            text.append(f', ')
        text.pop()
        text.append(f']\n\t\t= [')
        # Result
        for c in range(self.weight_vectors.shape[1]):
            text.append(f'{round(self.weight_vectors[winning_output_unit_idx][c] + self.learning_coef * vector[c], round_dec)}')
            text.append(f', ')
        text.pop()
        text.append(f']\n')

        # Normalize
        text.append(f'\t\t\tNormalize it:\n')
        text.append(f'\t\t\t\t[')
        for c in range(self.weight_vectors.shape[1]):
            text.append(f'{round(self.weight_vectors[winning_output_unit_idx][c] + self.learning_coef * vector[c], round_dec)}/{round((self.weight_vectors[winning_output_unit_idx] + self.learning_coef * vector).sum(), round_dec)}')
            text.append(f', ')
        text.pop()
        text.append(f']\n')
        text.append(f'\t\t\t\t= {np.round(self.normalize(self.weight_vectors[winning_output_unit_idx] + self.learning_coef * vector, True), round_dec)}\n\n')
        text.append(f'\t{self.training_labels[idx]} clusters under {self.output_unit_labels[winning_output_unit_idx]}\n\n')        
        print(''.join(text))
        # print(np.multiply(self.weight_vectors, vector).sum(axis = 1))

    def helper_print_clusters(self, clusters):
        for key, vals in clusters.items():
            print(f'{self.output_unit_labels[key]} cluster:\n{self.training_data[vals]}')
        print('\n\n')
#####################################################################################################

In [17]:
sclm = SCLM(
    learning_coef = 0.5,
    training_data = [
        [1,0,0,0,0,0],
        [1,0,0,1,0,0],
        [1,0,0,0,1,0],
        [0,0,1,1,0,1],
        [0,0,1,1,0,1],
        [0,0,1,0,0,1],
        [0,1,0,0,0,1],
        [0,1,0,0,0,1],
        [0,1,0,0,1,1],
        [1,0,0,0,0,0]
    ],
    testing_data = [
        [0,0,1,1,1,1],
        [1,0,0,0,1,1],
        [0,1,0,1,0,1]
    ]
)

sclm.init_rand_weights(num_input_units = 6, num_output_units = 3)
sclm.output_unit_labels = ['output_unit_P', 'output_unit_Q', 'output_unit_R']
print(sclm)

sclm.training_process()
sclm.testing_process()

learning_coef: 0.5
weight_vectors: 
	output_unit_P: 	[0.203  0.2624 0.1874 0.1265 0.033  0.1877]
	output_unit_Q: 	[0.1795 0.2072 0.1443 0.225  0.1996 0.0443]
	output_unit_R: 	[0.2409 0.0933 0.4482 0.0397 0.034  0.1439]
training_data: 
	training_data-1: 	[1 0 0 0 0 0]
	training_data-2: 	[1 0 0 1 0 0]
	training_data-3: 	[1 0 0 0 1 0]
	training_data-4: 	[0 0 1 1 0 1]
	training_data-5: 	[0 0 1 1 0 1]
	training_data-6: 	[0 0 1 0 0 1]
	training_data-7: 	[0 1 0 0 0 1]
	training_data-8: 	[0 1 0 0 0 1]
	training_data-9: 	[0 1 0 0 1 1]
	training_data-10: 	[1 0 0 0 0 0]
training_data: 
	testing_data-1: 	[0 0 1 1 1 1]
	testing_data-2: 	[1 0 0 0 1 1]
	testing_data-3: 	[0 1 0 1 0 1]

The Training Process: training_data-1: [1 0 0 0 0 0]
	Finding the winning output unit: 
		The net weighted sum of output_unit_P: (0.203*1) + (0.2624*0) + (0.1874*0) + (0.1265*0) + (0.033*0) + (0.1877*0) = 0.203
		The net weighted sum of output_unit_Q: (0.1795*1) + (0.2072*0) + (0.1443*0) + (0.225*0) + (0.1996*0) + (0.04

In [18]:
sclm = SCLM(
    learning_coef = 0.5,
    training_data = [
        [0,1,0,1]
    ],
    weight_vectors = [
        [.2,.4,.1,.3],
        [.5,.1,.2,.2],
        [.2,.1,.3,.4]
    ]
)

print(sclm)

sclm.training_process()

learning_coef: 0.5
weight_vectors: 
	output_unit-1: 	[0.2 0.4 0.1 0.3]
	output_unit-2: 	[0.5 0.1 0.2 0.2]
	output_unit-3: 	[0.2 0.1 0.3 0.4]
training_data: 
	training_data-1: 	[0 1 0 1]
training_data: 
	testing_data-1: 	[]

The Training Process: training_data-1: [0 1 0 1]
	Finding the winning output unit: 
		The net weighted sum of output_unit-1: (0.2*0) + (0.4*1) + (0.1*0) + (0.3*1) = 0.7
		The net weighted sum of output_unit-2: (0.5*0) + (0.1*1) + (0.2*0) + (0.2*1) = 0.3
		The net weighted sum of output_unit-3: (0.2*0) + (0.1*1) + (0.3*0) + (0.4*1) = 0.5

		The winning output unit is output_unit-1

	Updating the weight of the winning output:
		[0.2 + 0.5 * 0, 0.4 + 0.5 * 1, 0.1 + 0.5 * 0, 0.3 + 0.5 * 1]
		= [0.2, 0.9, 0.1, 0.8]
			Normalize it:
				[0.2/2.0, 0.9/2.0, 0.1/2.0, 0.8/2.0]
				= [0.1  0.45 0.05 0.4 ]

	training_data-1 clusters under output_unit-1


	Updated Weights: 
		output_unit-1: 	[0.1  0.45 0.05 0.4 ]
		output_unit-2: 	[0.5 0.1 0.2 0.2]
		output_unit-3: 	[0.2 0.1 0.3